In [16]:
import tensorflow
tensorflow.keras.__version__

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
import pandas as pd
df = pd.read_csv("HR_Mockdata.csv")
df = df.drop(columns=["sales \nfunction", "hire_date", 'last_evaluation', 'move_date', 'end_date',
                    'country_home', 'country_host', 
                       'left', 'fired_quit'])
# Drop null columns/rows
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.tail()

,#,satisfaction_level,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary,leader_host,culture_home,culture_host,function_host,business Size_host,economic Perspective_host,buzz_bust
14994,10244,1.0,2,105,2,0,0,medium,key talent,consensual,egalitarian,operations,united states,positive,buzz
14995,10326,1.0,4,254,3,1,0,medium,well placed talent,hierarquical,egalitarian,operations,united states,positive,buzz
14996,10918,1.0,4,240,4,0,0,low,well placed talent,hierarquical,egalitarian,operations,united states,positive,bust
14997,11376,1.0,5,167,3,0,0,medium,well placed talent,hierarquical,egalitarian,operations,united states,positive,buzz
14998,12828,1.0,3,170,3,0,0,medium,well placed talent,consensual,egalitarian,regulatory affairs,united states,positive,buzz


In [18]:
list(df.columns.values)

['#',
 'satisfaction_level',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'function_host',
 'business Size_host',
 'economic Perspective_host',
 'buzz_bust']

In [19]:
# select column names that need to be converted to dummy variables
to_dummify = [
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'economic Perspective_host',
 'business Size_host', 
 'function_host'
 ]

In [20]:
for dummy in to_dummify:
    df = pd.concat([df.drop(dummy, axis=1), pd.get_dummies(df[dummy], 
                prefix=dummy)], axis=1)
df.head()

,#,satisfaction_level,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,buzz_bust,salary_high,salary_low,...,function_host_communication,function_host_compliance,function_host_finance,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs
0,45,0.09,6,276,4,0,0,bust,0,1,...,0,0,0,0,0,0,1,0,0,0
1,513,0.09,6,259,4,0,0,buzz,0,1,...,0,0,0,0,0,0,0,0,0,0
2,529,0.09,6,301,5,0,0,buzz,0,0,...,0,0,0,0,0,0,0,0,0,0
3,689,0.09,6,257,4,0,0,buzz,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1189,0.09,5,279,4,0,0,buzz,0,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X = df.drop("buzz_bust", axis=1)
y = df["buzz_bust"]
print(X.shape, y.shape)
X.head(1)

(14999, 41) (14999,)


,#,satisfaction_level,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_high,salary_low,salary_medium,...,function_host_communication,function_host_compliance,function_host_finance,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs
0,45,0.09,6,276,4,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    random_state=1, stratify=y)

In [23]:
from sklearn.preprocessing import MinMaxScaler
# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

C:\Users\h4wth\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [24]:
# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [26]:

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=41))
model.add(Dense(units=2, activation='softmax'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6)                 252       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 14        
Total params: 266
Trainable params: 266
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 2s - loss: 0.5604 - acc: 0.7675
Epoch 2/100
 - 1s - loss: 0.5348 - acc: 0.7678
Epoch 3/100
 - 1s - loss: 0.5300 - acc: 0.7678
Epoch 4/100
 - 1s - loss: 0.5270 - acc: 0.7678
Epoch 5/100
 - 1s - loss: 0.5236 - acc: 0.7678
Epoch 6/100
 - 1s - loss: 0.5212 - acc: 0.7678
Epoch 7/100
 - 1s - loss: 0.5176 - acc: 0.7678
Epoch 8/100
 - 1s - loss: 0.5141 - acc: 0.7678
Epoch 9/100
 - 1s - loss: 0.5117 - acc: 0.7678
Epoch 10/100
 - 1s - loss: 0.5089 - acc: 0.7678
Epoch 11/100
 - 1s - loss: 0.5064 - acc: 0.7678
Epoch 12/100
 - 1s - loss: 0.5048 - acc: 0.7678
Epoch 13/100
 - 1s - loss: 0.5028 - acc: 0.7678
Epoch 14/100
 - 1s - loss: 0.5011 - acc: 0.7678
Epoch 15/100
 - 1s - loss: 0.4993 - acc: 0.7678
Epoch 16/100
 - 1s - loss: 0.4979 - acc: 0.7678
Epoch 17/100
 - 1s - loss: 0.4967 - acc: 0.7678
Epoch 18/100
 - 1s - loss: 0.4961 - acc: 0.7678
Epoch 19/100
 - 1s - loss: 0.4945 - acc: 0.7679
Epoch 20/100
 - 1s - loss: 0.4934 - acc: 0.7678
Epoch 21/100
 - 1s - loss: 0.4928 - acc: 0.7681
E

In [30]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 1s - loss: 0.4891 - acc: 0.7680
Normal Neural Network - Loss: 0.48905494354565937, Accuracy: 0.7680000066757202
